In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import plotly
plotly.tools.set_credentials_file(username='ninjamouse', api_key='LejqbK7NWhPU7zXoMuhw')

import numpy as np
import pandas as pd

df = pd.read_csv('first_attack.csv')

In [2]:
df.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,"2010-06-12 06:50:00,055616",88.208.248.162,192.168.4.120,HTTP,1514,Continuation
1,2,"2010-06-12 06:50:00,059867",88.208.248.162,192.168.4.120,HTTP,1514,Continuation
2,3,"2010-06-12 06:50:00,061151",192.168.4.120,88.208.248.162,TCP,60,2677 > 80 [ACK] Seq=1 Ack=2921 Win=17520 Len=0
3,4,"2010-06-12 06:50:00,062786",208.122.28.2,192.168.4.120,HTTP,1195,HTTP/1.1 200 OK (JPEG JFIF image)
4,5,"2010-06-12 06:50:00,064234",192.168.4.120,208.122.28.10,HTTP,140,GET /profile_images/159516/rollinsliar_small.j...


In [3]:
#df.sort_values(['Source', 'Destination'],  ascending=[True, False]  )
#df['Info'] = df['Info'].astype('str')
#get=df[df['Info'].str.contains('GET')]


#Sort the connections based on the pair Source-->Destination
#Select all the connections that have Source IP and Destiantion IP with 75.127.97.72
df.drop(['Time', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()
d=df[(df.Destination=='75.127.97.72') | (df.Source=='75.127.97.72')]   

In [4]:
df['Info'] = df['Info'].astype('str')
df[df['Info'].str.contains('GET')].head()

,No.,Time,Source,Destination,Protocol,Length,Info
4,5,"2010-06-12 06:50:00,064234",192.168.4.120,208.122.28.10,HTTP,140,GET /profile_images/159516/rollinsliar_small.j...
6,7,"2010-06-12 06:50:00,066804",192.168.4.120,88.208.248.162,HTTP,245,GET /Car-Products/Bike-Racks/Bike-Racks-Thule-...
10,11,"2010-06-12 06:50:00,092368",192.168.2.110,67.220.214.50,HTTP,112,GET /images/rss2.gif HTTP/1.1
12,13,"2010-06-12 06:50:00,121749",192.168.4.120,208.122.28.2,HTTP,149,GET /profile_images/228715/Logo_Baseball_Canad...
18,19,"2010-06-12 06:50:00,180490",192.168.4.120,208.122.28.4,HTTP,143,GET /profile_images/259115/discrimination_smal...


In [5]:
trace1=go.Scatter(x=d['Time'],y=d['Length'],mode='markers')
l=d.query('(Length>300)&(Length<600)')
trace2=go.Scatter(x=l['Time'],y=l['Length'], mode='lines')
data=[trace1, trace2]
py.iplot(data)

In [6]:
from sklearn import linear_model

In [7]:
X = df

# Treat the problem as a pure classification problem.

X['Protocol'] = X['Protocol'].astype('category')
# Could use float, but it doesn't make much sense
#X['Source'] = X['Source'].apply(lambda x: float(x.replace('.','')))

# Convert IPs to classes
X['Destination'] = X['Destination'].astype('category')
X['Source'] = X['Source'].astype('category')

# Convert info to classes as well
X['Info'] = X['Info'].astype('category')  # ToDo split info in relevant columns

# Drop the time series for now, just to play with a classifier
X = X.drop(['Time'], axis=1)
X.head()

,No.,Source,Destination,Protocol,Length,Info
0,1,88.208.248.162,192.168.4.120,HTTP,1514,Continuation
1,2,88.208.248.162,192.168.4.120,HTTP,1514,Continuation
2,3,192.168.4.120,88.208.248.162,TCP,60,2677 > 80 [ACK] Seq=1 Ack=2921 Win=17520 Len=0
3,4,208.122.28.2,192.168.4.120,HTTP,1195,HTTP/1.1 200 OK (JPEG JFIF image)
4,5,192.168.4.120,208.122.28.10,HTTP,140,GET /profile_images/159516/rollinsliar_small.j...


In [8]:
# Set some entries as 1 'attack' and 0 no attack
# The ones you defined in d above, are entries where an attack was plausible. Everything else is clean
Y = [0 if i not in l.index.values else 1 for i in df.index.values]

In [57]:
#1. Aici ce facem? Pentru fiecare valoare este asignata nu cod?
## Transformam toate coloanele care nu sunt numerice, in valori numerice.
## De exemplu, pentru coloana Protocol ai valori 'HTTP' si 'TCP'. Matematic e greu sa le analizezi
## daca nu sunt in format numeric. Asa ca functia lambda de mai jos iti transforma
## toate valorile 'HTTP' in 0 si 'TCP' in 1 (de exemplu). Daca ai mai multe coloane, ai mai multe valori (0,1,2,3..)
catColumns = X.select_dtypes(['category']).columns
X[catColumns] = X[catColumns].apply(lambda x: x.cat.codes)

In [9]:
X.columns

Index(['No.', 'Source', 'Destination', 'Protocol', 'Length', 'Info'], dtype='object')

In [10]:
#2. Banuiesc ca aici este aplicat algoritmul de linear regression? Ar trebui sa il inteleg sau doar il aplic si gata?
## Aici faci doar partea de model fitting a algoritmului. Pe baza valorilor din X, iti antrenezi modelul ca sa prezica
## valorile din Y. Aici calculezi coeficientii cei mai potriviti din formula liniara care iti satisfac cel mai bine
## X = Y
## sau mai in detaliu, ai avea 
## coef_Source * X['Source'] + coef_Destination * X['Destination'] + .. + coef_Info * X['Info'] = Y
## ,pentru fiecare linie din X si linia corespunzatoare din Y

clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X, Y)

ValueError: could not convert string to float: '2951  >  80 [ACK] Seq=262 Ack=101396 Win=17185 Len=0'

In [ ]:
# ## TODO

# df = pd.read_csv('second_attack.csv')

# X_test = df

# ## apply transformation methods as above

# pred = clf.predict(X_test)

In [ ]:
## TODO
## Compute 'd' the same way you computed it in the first case
## Plot d + pred
## Compare overalp between d and pred